In [1]:
import kagglehub
import subprocess

# Download latest version
path = kagglehub.dataset_download("awsaf49/brats20-dataset-training-validation")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/brats20-dataset-training-validation


In [2]:
!ls /kaggle/input/brats20-dataset-training-validation

BraTS2020_TrainingData	BraTS2020_ValidationData


In [28]:
!git clone https://github.com/JuneMetz/MedSegDiff.git
!mv MedSegDiff/* ./

Cloning into 'MedSegDiff'...
remote: Enumerating objects: 743, done.
remote: Counting objects: 100% (425/425), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 743 (delta 359), reused 250 (delta 250), pack-reused 318 (from 2)
Receiving objects: 100% (743/743), 3.92 MiB | 32.39 MiB/s, done.
Resolving deltas: 100% (415/415), done.
mv: cannot move 'MedSegDiff/data' to './data': Directory not empty
mv: cannot move 'MedSegDiff/scripts' to './scripts': Directory not empty


In [4]:
!mkdir /kaggle

mkdir: cannot create directory ‘/kaggle’: File exists


In [5]:
!mkdir /kaggle/working

In [6]:
!mkdir /kaggle/working/out

In [7]:
!cp -r /content/guided_diffusion /content/scripts/guided_diffusion

In [9]:
import os
import glob
import shutil
from tqdm import tqdm

def setup_brats_patient_structure():
    # Create the main directories
    base_dir = "/kaggle/working/brats"
    training_dir = os.path.join(base_dir, "training")
    validation_dir = os.path.join(base_dir, "validation")

    # Create directories if they don't exist
    os.makedirs(training_dir, exist_ok=True)
    os.makedirs(validation_dir, exist_ok=True)

    # Source directory with the copied BraTS data
    source_dir = path + "/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"

    # Get all patient folders
    patient_folders = sorted(glob.glob(os.path.join(source_dir, "BraTS20_Training_*")))

    # Use 80% for training, 20% for validation
    split_idx = int(len(patient_folders) * 0.8)
    training_patients = patient_folders[:split_idx]
    validation_patients = patient_folders[split_idx:]

    # Process training patients
    for i, patient_folder in enumerate(tqdm(training_patients, desc="Processing training data")):
        patient_id = os.path.basename(patient_folder).split('_')[-1]
        # Format patient_dir with a fixed width zero-padded number
        patient_dir = os.path.join(training_dir, f"patient_{i+1:02d}")
        os.makedirs(patient_dir, exist_ok=True)

        # Find all nifti files for this patient
        try:
            # List all files in the directory
            files = os.listdir(patient_folder)

            # Filter to get only the nifti files
            nifti_files = [f for f in files if f.endswith('.nii')]

            if not nifti_files:
                print(f"No .nii files found in {patient_folder}. Skipping.")
                continue

            # Process each modality
            for file in nifti_files:
                # Determine modality from filename
                if '_t1.' in file.lower():
                    modality = 't1'
                elif '_t1ce.' in file.lower():
                    modality = 't1ce'
                elif '_t2.' in file.lower():
                    modality = 't2'
                elif '_flair.' in file.lower():
                    modality = 'flair'
                elif '_seg.' in file.lower():
                    modality = 'seg'
                else:
                    print(f"Unknown modality in file {file}. Skipping.")
                    continue

                # Create new filename - fix the string formatting here
                # Use string format with zfill for zero-padding
                new_file = f"brats_train_{patient_id.zfill(3)}_{modality}_123_w.nii"

                # Copy and rename the file
                shutil.copy(
                    os.path.join(patient_folder, file),
                    os.path.join(patient_dir, new_file)
                )

            print(f"Processed patient {patient_id} to {patient_dir}")
        except Exception as e:
            print(f"Error processing {patient_folder}: {e}")

    # Process validation patients
    for i, patient_folder in enumerate(tqdm(validation_patients, desc="Processing validation data")):
        patient_id = os.path.basename(patient_folder).split('_')[-1]
        # Format patient_dir with a fixed width zero-padded number
        patient_dir = os.path.join(validation_dir, f"patient_{i+1:02d}")
        os.makedirs(patient_dir, exist_ok=True)

        # Find all nifti files for this patient
        try:
            # List all files in the directory
            files = os.listdir(patient_folder)

            # Filter to get only the nifti files
            nifti_files = [f for f in files if f.endswith('.nii')]

            if not nifti_files:
                print(f"No .nii files found in {patient_folder}. Skipping.")
                continue

            # Process each modality
            for file in nifti_files:
                # Determine modality from filename
                if '_t1.' in file.lower():
                    modality = 't1'
                elif '_t1ce.' in file.lower():
                    modality = 't1ce'
                elif '_t2.' in file.lower():
                    modality = 't2'
                elif '_flair.' in file.lower():
                    modality = 'flair'
                elif '_seg.' in file.lower():
                    modality = 'seg'
                else:
                    print(f"Unknown modality in file {file}. Skipping.")
                    continue

                # Create new filename - fix the string formatting here
                # Use string format with zfill for zero-padding
                new_file = f"brats_val_{patient_id.zfill(3)}_{modality}_123_w.nii"

                # Copy and rename the file
                shutil.copy(
                    os.path.join(patient_folder, file),
                    os.path.join(patient_dir, new_file)
                )

            print(f"Processed patient {patient_id} to {patient_dir}")
        except Exception as e:
            print(f"Error processing {patient_folder}: {e}")

    # Print summary
    num_training = len(os.listdir(training_dir))
    num_validation = len(os.listdir(validation_dir))

    print(f"Successfully created the dataset structure:")
    print(f"Training patients: {num_training}")
    print(f"Validation patients: {num_validation}")
    print(f"Data located at: {base_dir}")

    return base_dir

# Call the function to set up the data structure
brats_data_dir = setup_brats_patient_structure()

Processing training data:   1%|          | 2/295 [00:00<00:46,  6.25it/s]

Processed patient 001 to /kaggle/working/brats/training/patient_01
Processed patient 002 to /kaggle/working/brats/training/patient_02


Processing training data:   1%|          | 3/295 [00:01<02:00,  2.43it/s]

Processed patient 003 to /kaggle/working/brats/training/patient_03


Processing training data:   1%|▏         | 4/295 [00:01<02:37,  1.85it/s]

...

Processed patient 367 to /kaggle/working/brats/validation/patient_72


Processing validation data:  99%|█████████▊| 73/74 [01:30<00:01,  1.49s/it]

Processed patient 368 to /kaggle/working/brats/validation/patient_73


Processing validation data: 100%|██████████| 74/74 [01:31<00:00,  1.24s/it]

Processed patient 369 to /kaggle/working/brats/validation/patient_74
Successfully created the dataset structure:
Training patients: 295
Validation patients: 74
Data located at: /kaggle/working/brats


In [10]:
!pip install blobfile
!pip install nibabel
!pip install visdom
!pip install torchsummary
!pip install batchgenerators
!pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408195 sha256=cdcb4b256de65467a26850ff83ea8c3ad4b6cbcc6c21b300fb3f7e5ebb8222e8
  Stored in directory: /root/.cache/pip/wheels/fa/a4/bb/2be445c295d88a74f9c0a4232f04860ca489a5c7c57eb959d9
Successfully built visdom
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 6.7 MB/s eta 0:00:00
  Created wheel for batchgenerators: filename=batchgenerators-0.25.1-py3-none-any.whl size=93088 sha256=0a89300a399114f122fa91eedbeed967651e337071b2a3d5b1057a152e268f47
  Stored in directory: /root/.cache/pip/wheels/56/11/c7/fadca30e054c6020

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4458271 sha256=a81bcf62c869dc754d621bc79b35d82fa0ffeb420344a942e01fc569dd645d3a
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


In [11]:
import pdb

In [29]:
import sys
import os
sys.path.append("/content/scripts")
sys.path.append("./")
from ssl import OP_NO_TLSv1
import nibabel as nib
import sys
import random
import numpy as np
import time
from PIL import Image
import torch.distributed as dist
import torchvision.utils as vutils
from guided_diffusion import dist_util, logger
from guided_diffusion.resample import create_named_schedule_sampler
from guided_diffusion.bratsloader import BRATSDataset, BRATSDataset3D
from guided_diffusion.isicloader import ISICDataset
from guided_diffusion.custom_dataset_loader import CustomDataset
from guided_diffusion.script_util import (
    NUM_CLASSES,
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    args_to_dict,
    add_dict_to_argparser,
)
import torch as th
import torch.nn as nn
from guided_diffusion.train_util import TrainLoop
from guided_diffusion.utils import staple
import torchvision.transforms as transforms
from torchsummary import summary

from dataclasses import dataclass

@dataclass
class Args:
  data_name: str
  data_dir: str
  out_dir: str
  image_size: int
  num_channels: int
  class_cond: bool
  num_res_blocks: int
  num_heads: int
  learn_sigma: bool
  use_scale_shift_norm: bool
  attention_resolutions: str
  diffusion_steps: int
  noise_schedule: str
  rescale_learned_sigmas: bool
  rescale_timesteps: bool
  lr: float
  batch_size: int
  patience: int
  min_delta: float
  microbatch: int
  ema_rate: str
  log_interval: int
  save_interval: int
  resume_checkpoint: str
  schedule_sampler: str
  weight_decay: float
  lr_anneal_steps: int
  use_fp16: bool
  fp16_scale_growth: float
  gpu_dev: str
  multi_gpu: str
  in_ch: int
  num_heads_upsample: int
  num_head_channels: int
  resblock_updown: bool
  dpm_solver: bool
  version: str
  channel_mult: str
  dropout: float
  use_checkpoint: bool
  use_new_attention_order: bool
  timestep_respacing: str
  use_kl: bool
  predict_xstart: bool
  model_path: str
  num_ensemble: int
  use_ddim: bool
  clip_denoised: bool
  num_samples: int
  activation_function: object
  final_nonlin: object

argsdict = dict(
  data_name = 'BRATS',
  data_dir = "/kaggle/working/brats",
  out_dir = "/kaggle/working/out",
  image_size = 256,
  num_channels = 128,
  class_cond = False,
  num_res_blocks = 2,
  num_heads = 1,
  learn_sigma = True,
  use_scale_shift_norm = False,
  attention_resolutions = "16",
  diffusion_steps = 1000,
  noise_schedule = 'linear',
  rescale_learned_sigmas = False,
  rescale_timesteps = False,
  lr = 1e-4,
  batch_size = 4,
  patience = 2,
  min_delta = 0.001,
  microbatch = -1,  # -1 disables microbatches
  ema_rate = "0.9999",  # comma-separated list of EMA values
  log_interval = 100,
  save_interval = 5000,
  resume_checkpoint = None, #"/results/pretrainedmodel.pt"
  schedule_sampler="uniform",
  weight_decay=0.0,
  lr_anneal_steps=400, # used for early stopping
  use_fp16=False,
  fp16_scale_growth=1e-3,
  gpu_dev = "0",
  multi_gpu = None, #"0,1,2"
  activation_function = nn.ReLU, # default is nn.SiLU
  final_nonlin = nn.Softmax, # default is sigmoid_helper (i.e. sigmoid)

  # useful for sampling only:
  model_path = "/kaggle/working/out/savedmodel000400.pt",
  num_ensemble = 5,
  use_ddim = False, #originally False
  clip_denoised = True,
  num_samples = 5,
)
argsdict.update(model_and_diffusion_defaults())

args = Args(**argsdict)

seed=10
th.manual_seed(seed)
th.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

In [22]:
# See BRATS Data Structure
!ls /kaggle/working/brats/training/patient_01

brats_train_001_flair_123_w.nii  brats_train_001_t1ce_123_w.nii
brats_train_001_seg_123_w.nii	 brats_train_001_t2_123_w.nii
brats_train_001_t1_123_w.nii


In [23]:
def custom_brats_loader():
    """Create a custom BraTS dataset loader that adapts to your data structure"""
    import os
    import glob
    import nibabel as nib
    import torch
    import numpy as np
    import matplotlib.pyplot as plt
    from torch.utils.data import Dataset, DataLoader
    import torchvision.transforms as transforms

    class CustomBratsDataset(Dataset):
        def __init__(self, data_dir, image_size=256, mode='train'):
            self.data_dir = data_dir
            self.mode = mode
            self.image_size = image_size
            self.transform = transforms.Compose([
                transforms.Resize((image_size, image_size)),
                transforms.ToTensor()
            ])

            # First check what files are available
            print(f"Scanning {data_dir} for BraTS data...")
            all_files = glob.glob(os.path.join(data_dir, "**/*.*"), recursive=True)
            print(f"Found {len(all_files)} total files")

            # Print some files to see naming pattern
            for file in all_files[:5]:
                print(f"Sample file: {file}")

            # Find all modalities available
            modalities = set()
            for file in all_files:
                filename = os.path.basename(file).lower()
                if 't1.' in filename or '_t1_' in filename:
                    modalities.add('t1')
                if 't1ce' in filename or 't1gd' in filename or 't1+c' in filename:
                    modalities.add('t1ce')
                if 't2.' in filename or '_t2_' in filename:
                    modalities.add('t2')
                if 'flair' in filename:
                    modalities.add('flair')
                if 'seg' in filename or 'mask' in filename or 'label' in filename:
                    modalities.add('seg')

            print(f"Detected modalities: {modalities}")

            # Group files by patient/case
            self.samples = []

            # Pattern matching approach - adapt based on your file naming
            for root, dirs, files in os.walk(data_dir):
                case_files = {}

                for file in files:
                    filepath = os.path.join(root, file)
                    filename = file.lower()

                    # Check if this is a NIfTI file
                    if not (filename.endswith('.nii') or filename.endswith('.nii.gz')):
                        continue

                    # Detect modality from filename
                    modality = None
                    if 't1ce' in filename or 't1gd' in filename or 't1+c' in filename:
                        modality = 't1ce'
                    elif 't1.' in filename or '_t1_' in filename:
                        modality = 't1'
                    elif 't2.' in filename or '_t2_' in filename:
                        modality = 't2'
                    elif 'flair' in filename:
                        modality = 'flair'
                    elif 'seg' in filename or 'mask' in filename or 'label' in filename:
                        modality = 'seg'

                    if modality:
                        case_files[modality] = filepath

                # Add to samples if we have at least 3 modalities
                if len(case_files) >= 3:
                    self.samples.append(case_files)

            print(f"Found {len(self.samples)} valid BraTS cases")

            # Show what modalities each case has
            for i, case in enumerate(self.samples[:3]):
                print(f"Case {i+1} has modalities: {list(case.keys())}")

        def __len__(self):
            return len(self.samples)

        def __getitem__(self, idx):
            case = self.samples[idx]

            # Initialize input tensor with zeros (4 modalities)
            input_tensor = torch.zeros((4, self.image_size, self.image_size))
            mask_tensor = torch.zeros((1, self.image_size, self.image_size))

            # Map modalities to tensor indices
            modality_map = {'t1': 0, 't1ce': 1, 't2': 2, 'flair': 3}

            # Load each available modality
            for modality, path in case.items():
                if modality in modality_map:
                    try:
                        # Load NIfTI file
                        nii_img = nib.load(path)
                        data = nii_img.get_fdata()

                        # For simplicity, take a middle slice
                        middle_slice = data.shape[2] // 2
                        slice_data = data[:, :, middle_slice]

                        # Normalize slice
                        slice_data = (slice_data - slice_data.min()) / (slice_data.max() - slice_data.min() + 1e-8)

                        # Resize to expected dimensions
                        slice_tensor = torch.tensor(slice_data).float()
                        slice_tensor = transforms.Resize((self.image_size, self.image_size))(slice_tensor.unsqueeze(0)).squeeze(0)

                        # Place in the correct channel
                        input_tensor[modality_map[modality]] = slice_tensor
                    except Exception as e:
                        print(f"Error loading {modality} for case {idx}: {e}")

                # Load segmentation mask if available
                elif modality == 'seg':
                    try:
                        nii_img = nib.load(path)
                        data = nii_img.get_fdata()

                        # Take same middle slice
                        middle_slice = data.shape[2] // 2
                        mask_data = data[:, :, middle_slice]

                        # Binarize mask (any value > 0 is considered part of tumor)
                        mask_data = (mask_data > 0).astype(np.float32)

                        # Resize to expected dimensions
                        mask_tensor = torch.tensor(mask_data).float()
                        mask_tensor = transforms.Resize((self.image_size, self.image_size))(mask_tensor.unsqueeze(0))
                    except Exception as e:
                        print(f"Error loading segmentation for case {idx}: {e}")

            return input_tensor, mask_tensor, str(case.get('t1', list(case.values())[0]))

    # Create and return the custom dataset
    dataset = CustomBratsDataset(args.data_dir, image_size=args.image_size)

    # Show a sample
    if len(dataset) > 0:
        sample_input, sample_mask, sample_path = dataset[0]

        plt.figure(figsize=(15, 8))
        plt.subplot(2, 3, 1)
        plt.imshow(sample_input[0], cmap='gray')
        plt.title('T1')
        plt.axis('off')

        plt.subplot(2, 3, 2)
        plt.imshow(sample_input[1], cmap='gray')
        plt.title('T1CE')
        plt.axis('off')

        plt.subplot(2, 3, 3)
        plt.imshow(sample_input[2], cmap='gray')
        plt.title('T2')
        plt.axis('off')

        plt.subplot(2, 3, 4)
        plt.imshow(sample_input[3], cmap='gray')
        plt.title('FLAIR')
        plt.axis('off')

        plt.subplot(2, 3, 5)
        plt.imshow(sample_mask[0], cmap='gray')
        plt.title('Segmentation Mask')
        plt.axis('off')

        plt.tight_layout()
        plt.savefig('/kaggle/working/brats_sample.png')
        plt.close()

    return dataset

In [24]:
## Train with Save Function

def train_with_custom_loader():
    """Training function that uses our custom BraTS loader"""
    dist_util.setup_dist(args)
    logger.configure(dir = args.out_dir)
    logger.log("Creating data loader with custom approach...")

    if args.data_name == 'ISIC':
        # Original ISIC loading code
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor(),]
        transform_train = transforms.Compose(tran_list)
        ds = ISICDataset(args, args.data_dir, transform_train)
        args.in_ch = 4
    elif args.data_name == 'BRATS':
        # Use our custom loader instead
        ds = custom_brats_loader()
        args.in_ch = 5 # 4 MRI modalities + 1 noise channel
    else:
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor(),]
        transform_train = transforms.Compose(tran_list)
        print("Your current directory : ", args.data_dir)
        ds = CustomDataset(args, args.data_dir, transform_train)
        args.in_ch = 4

    # Check if we have a valid dataset
    if len(ds) == 0:
        logger.log("No valid data samples found. Please check your data directory.")
        return

    logger.log(f"Dataset loaded with {len(ds)} samples")

    # Set batch size to 1 if dataset is very small
    if len(ds) < args.batch_size:
        logger.log(f"Dataset has {len(ds)} samples, reducing batch size to 1")
        args.batch_size = 1

    datal = th.utils.data.DataLoader(
        ds,
        batch_size=args.batch_size,
        shuffle=True)
    data = iter(datal)

    logger.log("Creating model and diffusion...")
    model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, model_and_diffusion_defaults().keys())
    )

    if args.multi_gpu:
        model = th.nn.DataParallel(model,device_ids=[int(id) for id in args.multi_gpu.split(',')])
        model.to(device = th.device('cuda', int(args.gpu_dev)))
    else:
        model.to(dist_util.dev())

    schedule_sampler = create_named_schedule_sampler(args.schedule_sampler, diffusion, maxt=args.diffusion_steps)

    logger.log("Training model...")

    # Add try-except to catch and diagnose any errors during training
    try:
        TrainLoop(
            model=model,
            diffusion=diffusion,
            classifier=None,
            data=data,
            dataloader=datal,
            batch_size=args.batch_size,
            microbatch=args.microbatch,
            lr=args.lr,
            ema_rate=args.ema_rate,
            log_interval=args.log_interval,
            save_interval=args.save_interval,
            resume_checkpoint=args.resume_checkpoint,
            use_fp16=args.use_fp16,
            fp16_scale_growth=args.fp16_scale_growth,
            schedule_sampler=schedule_sampler,
            weight_decay=args.weight_decay,
            lr_anneal_steps=args.lr_anneal_steps,
        ).run_loop()

    except Exception as e:
        logger.log(f"Error during training: {e}")
        import traceback
        traceback.print_exc()

        # Try to save model before exiting
        if hasattr(model, 'module'):
            th.save(model.module.state_dict(), os.path.join(args.out_dir, "model_error_checkpoint.pt"))
        else:
            th.save(model.state_dict(), os.path.join(args.out_dir, "model_error_checkpoint.pt"))
        logger.log("Saved model checkpoint despite error")

    # Save to Google Drive
    save_model_to_drive(model, args)

# Function to save model to Google Drive
def save_model_to_drive(model, args):
    """
    Save the model to Google Drive to ensure it persists outside the Colab environment
    """
    logger.log("Saving model to Google Drive...")

    try:
        # First check if Google Drive is mounted
        from google.colab import drive
        import os

        # Check if drive is already mounted
        drive_mounted = os.path.exists('/content/drive')

        if not drive_mounted:
            logger.log("Mounting Google Drive...")
            drive.mount('/content/drive')

        # Create directory for model in Google Drive if it doesn't exist
        save_dir = f'/content/drive/MyDrive/medsegdiff_models/{args.data_name}'
        os.makedirs(save_dir, exist_ok=True)

        # Generate a timestamp for the filename
        from datetime import datetime
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        # Save the model
        if hasattr(model, 'module'):
            model_path = os.path.join(save_dir, f"{args.data_name}_model_{timestamp}.pt")
            th.save(model.module.state_dict(), model_path)
        else:
            model_path = os.path.join(save_dir, f"{args.data_name}_model_{timestamp}.pt")
            th.save(model.state_dict(), model_path)

        # Also save a copy of the configuration
        import json
        config_path = os.path.join(save_dir, f"{args.data_name}_config_{timestamp}.json")
        with open(config_path, 'w') as f:
            # Convert args to dictionary
            config_dict = vars(args)
            # Convert any non-serializable objects to strings
            for key, value in config_dict.items():
                if not isinstance(value, (str, int, float, bool, list, dict, type(None))):
                    config_dict[key] = str(value)
            json.dump(config_dict, f, indent=4)

        logger.log(f"Model saved successfully to Google Drive at: {model_path}")
        logger.log(f"Config saved at: {config_path}")

        # Create a convenience symlink to the latest model
        latest_model_path = os.path.join(save_dir, f"{args.data_name}_model_latest.pt")
        latest_config_path = os.path.join(save_dir, f"{args.data_name}_config_latest.json")

        # Remove existing symlinks if they exist
        if os.path.exists(latest_model_path):
            os.remove(latest_model_path)
        if os.path.exists(latest_config_path):
            os.remove(latest_config_path)

        # Create new symlinks
        os.symlink(model_path, latest_model_path)
        os.symlink(config_path, latest_config_path)

        logger.log(f"Created symlinks for easy access to latest model and config")

        return model_path

    except Exception as e:
        logger.log(f"Error saving model to Google Drive: {e}")
        import traceback
        traceback.print_exc()

        # Try to save locally as fallback
        local_path = os.path.join(args.out_dir, f"{args.data_name}_model_local.pt")
        if hasattr(model, 'module'):
            th.save(model.module.state_dict(), local_path)
        else:
            th.save(model.state_dict(), local_path)
        logger.log(f"Model could not be saved to Drive, but was saved locally at: {local_path}")
        logger.log("You should manually copy this file to persistent storage before the Colab session ends")

        return local_path

# Add a function to load a saved model from Google Drive
def load_model_from_drive(model, model_path=None, data_name=None):
    """
    Load a saved model from Google Drive

    Args:
        model: The model to load weights into
        model_path: Specific path to the model file, if None will use latest
        data_name: Type of data model was trained on (BRATS, ISIC, etc.)

    Returns:
        The model with loaded weights
    """
    logger.log("Loading model from Google Drive...")

    try:
        # First check if Google Drive is mounted
        from google.colab import drive
        import os

        # Check if drive is already mounted
        drive_mounted = os.path.exists('/content/drive')

        if not drive_mounted:
            logger.log("Mounting Google Drive...")
            drive.mount('/content/drive')

        # If no specific path is given, try to load the latest model
        if model_path is None:
            if data_name is None:
                raise ValueError("Either model_path or data_name must be provided")

            model_path = f'/content/drive/MyDrive/medsegdiff_models/{data_name}/{data_name}_model_latest.pt'

        # Check if the model exists
        if not os.path.exists(model_path):
            logger.log(f"Model not found at {model_path}")
            return None

        # Load the state dict
        state_dict = th.load(model_path, map_location='cpu')

        # Process the state dict to handle DataParallel models
        from collections import OrderedDict
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            # Remove 'module.' if present and model is not DataParallel
            if k.startswith('module.') and not isinstance(model, th.nn.DataParallel):
                new_state_dict[k[7:]] = v
            # Add 'module.' if not present and model is DataParallel
            elif not k.startswith('module.') and isinstance(model, th.nn.DataParallel):
                new_state_dict[f'module.{k}'] = v
            else:
                new_state_dict[k] = v

        # Load the weights
        model.load_state_dict(new_state_dict)
        logger.log(f"Model loaded successfully from {model_path}")

        return model

    except Exception as e:
        logger.log(f"Error loading model from Google Drive: {e}")
        import traceback
        traceback.print_exc()
        return None

In [30]:
train_with_custom_loader()

Logging to /kaggle/working/out
Creating data loader with custom approach...
Scanning /kaggle/working/brats for BraTS data...
Found 1844 total files
Sample file: /kaggle/working/brats/validation/patient_11/brats_val_306_t1_123_w.nii
Sample file: /kaggle/working/brats/validation/patient_11/brats_val_306_t1ce_123_w.nii
Sample file: /kaggle/working/brats/validation/patient_11/brats_val_306_seg_123_w.nii
Sample file: /kaggle/working/brats/validation/patient_11/brats_val_306_flair_123_w.nii
Sample file: /kaggle/working/brats/validation/patient_11/brats_val_306_t2_123_w.nii
Detected modalities: {'t1', 'seg', 't2', 't1ce', 'flair'}
Found 369 valid BraTS cases
Case 1 has modalities: ['t1', 't1ce', 'seg', 'flair', 't2']
Case 2 has modalities: ['t2', 'flair', 't1ce', 't1', 'seg']
Case 3 has modalities: ['t1ce', 't2', 'flair', 't1', 'seg']
Dataset loaded with 369 samples
Creating model and diffusion...
Training model...
---------------------------
| grad_norm    | 10.9     |
| loss         | 1    

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 944, in save
    _save(
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 1216, in _save
    zip_file.write_record(name, storage, num_bytes)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-24-4592c44b1b0c>", line 79, in train_with_custom_loader
    ).run_loop()
      ^^^^^^^^^^
  File "/content/guided_diffusion/train_util.py", line 194, in run_loop
    self.save()
  File "/content/guided_diffusion/train_util.py", line 292, in save
    save_checkpoint(rate, params)
  File "/content/guided_diffusion/train_util.py", line 288, in save_checkpoint
    th.save(state_dict, f)
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 943, in save
    with _open_zipfile_writer(f) as opened_zipfile:
  File "/usr/local/lib/python3.11/dist-pack

KeyboardInterrupt: 

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')

    # Create a folder in Google Drive
    import os
    os.makedirs('/content/drive/MyDrive/brats_models', exist_ok=True)

    # Copy the model file to Google Drive
    os.system('cp /kaggle/working/out/BRATS_model_local.pt /content/drive/MyDrive/brats_models/')

    print("Model saved to Google Drive at: /content/drive/MyDrive/brats_models/BRATS_model_local.pt")
except:
    print("This method only works in Google Colab. Please use another method.")

In [ ]:
# OLD - Wrong Dataloading...

def visualize(img):
    _min = img.min()
    _max = img.max()
    normalized_img = (img - _min)/ (_max - _min)
    return normalized_img

def sample():
    dist_util.setup_dist(args)
    logger.configure(dir = args.out_dir)

    if args.data_name == 'ISIC':
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor(),]
        transform_test = transforms.Compose(tran_list)

        ds = ISICDataset(args, args.data_dir, transform_test, mode = 'Test')
        args.in_ch = 4
    elif args.data_name == 'BRATS':
        tran_list = [transforms.Resize((args.image_size,args.image_size)),]
        transform_test = transforms.Compose(tran_list)

        ds = BRATSDataset3D(args.data_dir,transform_test)
        args.in_ch = 5
    else:
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor()]
        transform_test = transforms.Compose(tran_list)

        ds = CustomDataset(args, args.data_dir, transform_test, mode = 'Test')
        args.in_ch = 4

    datal = th.utils.data.DataLoader(
        ds,
        batch_size=args.batch_size,
        shuffle=True)
    data = iter(datal)

    logger.log("creating model and diffusion...")

    model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, model_and_diffusion_defaults().keys())
    )
    all_images = []


    state_dict = dist_util.load_state_dict(args.model_path, map_location="cpu")

    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        # name = k[7:] # remove `module.`
        if 'module.' in k:
            new_state_dict[k[7:]] = v
            # load params
        else:
            new_state_dict = state_dict

    model.load_state_dict(new_state_dict)

    model.to(dist_util.dev())
    if args.use_fp16:
        model.convert_to_fp16()
    model.eval()
    for _ in range(args.num_samples):
        b, m, path = next(data)  #should return an image from the dataloader "data"
        c = th.randn_like(b[:, :1, ...])
        img = th.cat((b, c), dim=1)     #add a noise channel$
        if args.data_name == 'ISIC':
            slice_ID=path[0].split("_")[-1].split('.')[0]
        elif args.data_name == 'BRATS':
            # slice_ID=path[0].split("_")[2] + "_" + path[0].split("_")[4]
            slice_ID=path[0].split("_")[-3] + "_" + path[0].split("slice")[-1].split('.nii')[0]

        logger.log("sampling...")

        start = th.cuda.Event(enable_timing=True)
        end = th.cuda.Event(enable_timing=True)
        enslist = []

        for i in range(args.num_ensemble):  #this is for the generation of an ensemble of 5 masks.
            model_kwargs = {}
            start.record()
            sample_fn = (
                diffusion.p_sample_loop_known if not args.use_ddim else diffusion.ddim_sample_loop_known
            )
            # pdb.set_trace() #imbedded debug !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            # truncating model output after its internally calculated in dim 2 should fix the issue:
            # https://github.com/SuperMedIntel/MedSegDiff/issues/201
            # fixable by just editing gaussian_diffusion.py:346 to be permissive
            sample, x_noisy, org, cal, cal_out = sample_fn( # broken
                model,
                (args.batch_size, 3, args.image_size, args.image_size), img,
                step = args.diffusion_steps,
                clip_denoised=args.clip_denoised,
                model_kwargs=model_kwargs,
            )

            end.record()
            th.cuda.synchronize()
            print('time for 1 sample', start.elapsed_time(end))  #time measurement for the generation of 1 sample

            co = th.tensor(cal_out)
            if args.version == 'new':
                enslist.append(sample[:,-1,:,:])
            else:
                enslist.append(co)

            if args.debug:
                # print('sample size is',sample.size())
                # print('org size is',org.size())
                # print('cal size is',cal.size())
                if args.data_name == 'ISIC':
                    # s = th.tensor(sample)[:,-1,:,:].unsqueeze(1).repeat(1, 3, 1, 1)
                    o = th.tensor(org)[:,:-1,:,:]
                    c = th.tensor(cal).repeat(1, 3, 1, 1)
                    # co = co.repeat(1, 3, 1, 1)

                    s = sample[:,-1,:,:]
                    b,h,w = s.size()
                    ss = s.clone()
                    ss = ss.view(s.size(0), -1)
                    ss -= ss.min(1, keepdim=True)[0]
                    ss /= ss.max(1, keepdim=True)[0]
                    ss = ss.view(b, h, w)
                    ss = ss.unsqueeze(1).repeat(1, 3, 1, 1)

                    tup = (ss,o,c)
                elif args.data_name == 'BRATS':
                    s = th.tensor(sample)[:,-1,:,:].unsqueeze(1)
                    m = th.tensor(m.to(device = 'cuda:0'))[:,0,:,:].unsqueeze(1)
                    o1 = th.tensor(org)[:,0,:,:].unsqueeze(1)
                    o2 = th.tensor(org)[:,1,:,:].unsqueeze(1)
                    o3 = th.tensor(org)[:,2,:,:].unsqueeze(1)
                    o4 = th.tensor(org)[:,3,:,:].unsqueeze(1)
                    c = th.tensor(cal)

                    tup = (o1/o1.max(),o2/o2.max(),o3/o3.max(),o4/o4.max(),m,s,c,co)

                compose = th.cat(tup,0)
                vutils.save_image(compose, fp = os.path.join(args.out_dir, str(slice_ID)+'_output'+str(i)+".png"), nrow = 1, padding = 10)
        ensres = staple(th.stack(enslist,dim=0)).squeeze(0)
        vutils.save_image(ensres, fp = os.path.join(args.out_dir, str(slice_ID)+'_output_ens'+".png"), nrow = 1, padding = 10)

In [ ]:
def visualize(img):
    _min = img.min()
    _max = img.max()
    normalized_img = (img - _min)/ (_max - _min)
    return normalized_img

In [ ]:
def sample():
    dist_util.setup_dist(args)
    logger.configure(dir=args.out_dir)

    # Replace the original BRATs dataset loading with your custom loader
    if args.data_name == 'ISIC':
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor(),]
        transform_test = transforms.Compose(tran_list)

        ds = ISICDataset(args, args.data_dir, transform_test, mode='Test')
        args.in_ch = 4
    elif args.data_name == 'BRATS':
        # Use the custom BraTS dataset loader defined in the second file
        ds = custom_brats_loader()
        args.in_ch = 5  # Keep the original channel configuration
    else:
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor()]
        transform_test = transforms.Compose(tran_list)

        ds = CustomDataset(args, args.data_dir, transform_test, mode='Test')
        args.in_ch = 4

    # Set default values for missing attributes
    if not hasattr(args, 'num_samples'):
        args.num_samples = 5  # Default to 5 samples if not specified
        logger.log(f"num_samples not specified, defaulting to {args.num_samples}")

    if not hasattr(args, 'num_ensemble'):
        args.num_ensemble = 5  # Default to 5 ensemble members if not specified
        logger.log(f"num_ensemble not specified, defaulting to {args.num_ensemble}")

    if not hasattr(args, 'version'):
        args.version = 'new'  # Default to 'new' version if not specified
        logger.log(f"version not specified, defaulting to '{args.version}'")

    if not hasattr(args, 'debug'):
        args.debug = True  # Default to debug mode being on
        logger.log(f"debug not specified, defaulting to {args.debug}")

    datal = th.utils.data.DataLoader(
        ds,
        batch_size=args.batch_size,
        shuffle=True)
    data = iter(datal)

    logger.log("creating model and diffusion...")

    model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, model_and_diffusion_defaults().keys())
    )
    all_images = []

    state_dict = dist_util.load_state_dict(args.model_path, map_location="cpu")

    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        if 'module.' in k:
            new_state_dict[k[7:]] = v
        else:
            new_state_dict = state_dict

    model.load_state_dict(new_state_dict)

    model.to(dist_util.dev())
    if args.use_fp16:
        model.convert_to_fp16()
    model.eval()

    # Use try/except to handle any issues
    try:
        for _ in range(args.num_samples):
            b, m, path = next(data)  # should return an image from the dataloader "data"
            c = th.randn_like(b[:, :1, ...])
            img = th.cat((b, c), dim=1)  # add a noise channel

            # Adjust the slice_ID extraction to match the format from your custom loader
            if args.data_name == 'ISIC':
                slice_ID = path[0].split("_")[-1].split('.')[0]
            elif args.data_name == 'BRATS':
                # Extract a more simplified ID from the path
                # This extracts just the base filename without the extension
                slice_ID = os.path.basename(path[0]).split('.')[0]

            logger.log("sampling...")

            start = th.cuda.Event(enable_timing=True)
            end = th.cuda.Event(enable_timing=True)
            enslist = []

            for i in range(args.num_ensemble):  # this is for the generation of an ensemble of masks
                model_kwargs = {}
                start.record()
                sample_fn = (
                    diffusion.p_sample_loop_known if not args.use_ddim else diffusion.ddim_sample_loop_known
                )

                sample, x_noisy, org, cal, cal_out = sample_fn(
                    model,
                    (args.batch_size, 3, args.image_size, args.image_size), img,
                    step=args.diffusion_steps,
                    clip_denoised=args.clip_denoised,
                    model_kwargs=model_kwargs,
                )

                end.record()
                th.cuda.synchronize()
                print('time for 1 sample', start.elapsed_time(end))

                co = th.tensor(cal_out)
                if args.version == 'new':
                    enslist.append(sample[:,-1,:,:])
                else:
                    enslist.append(co)

                if args.debug:
                    if args.data_name == 'BRATS':
                        # Adjust the debug visualization for your custom loader format
                        s = th.tensor(sample)[:,-1,:,:].unsqueeze(1)
                        m_tensor = th.tensor(m.to(device='cuda:0'))

                        # Make sure m_tensor is the right shape
                        if len(m_tensor.shape) == 4:
                            m_display = m_tensor[:,0,:,:].unsqueeze(1)
                        else:
                            m_display = m_tensor  # Assuming it's already [B,1,H,W]

                        # Display the 4 modalities from org
                        o1 = th.tensor(org)[:,0,:,:].unsqueeze(1)
                        o2 = th.tensor(org)[:,1,:,:].unsqueeze(1)
                        o3 = th.tensor(org)[:,2,:,:].unsqueeze(1)
                        o4 = th.tensor(org)[:,3,:,:].unsqueeze(1)
                        c = th.tensor(cal)

                        # Normalize for better visualization
                        o1_norm = o1/o1.max() if o1.max() > 0 else o1
                        o2_norm = o2/o2.max() if o2.max() > 0 else o2
                        o3_norm = o3/o3.max() if o3.max() > 0 else o3
                        o4_norm = o4/o4.max() if o4.max() > 0 else o4

                        tup = (o1_norm, o2_norm, o3_norm, o4_norm, m_display, s, c, co)
                        compose = th.cat(tup, 0)
                        vutils.save_image(compose, fp=os.path.join(args.out_dir, f"{slice_ID}_output{i}.png"), nrow=1, padding=10)

                    # Keep the original ISIC visualization code
                    elif args.data_name == 'ISIC':
                        o = th.tensor(org)[:,:-1,:,:]
                        c = th.tensor(cal).repeat(1, 3, 1, 1)

                        s = sample[:,-1,:,:]
                        b,h,w = s.size()
                        ss = s.clone()
                        ss = ss.view(s.size(0), -1)
                        ss -= ss.min(1, keepdim=True)[0]
                        ss /= ss.max(1, keepdim=True)[0]
                        ss = ss.view(b, h, w)
                        ss = ss.unsqueeze(1).repeat(1, 3, 1, 1)

                        tup = (ss, o, c)
                        compose = th.cat(tup, 0)
                        vutils.save_image(compose, fp=os.path.join(args.out_dir, f"{slice_ID}_output{i}.png"), nrow=1, padding=10)

            # Check that staple function exists before calling it
            if 'staple' in globals():
                # Generate the ensemble result
                ensres = staple(th.stack(enslist, dim=0)).squeeze(0)
                vutils.save_image(ensres, fp=os.path.join(args.out_dir, f"{slice_ID}_output_ens.png"), nrow=1, padding=10)
            else:
                # If staple function doesn't exist, use average instead
                logger.log("staple function not found, using simple average for ensemble")
                ensres = th.mean(th.stack(enslist, dim=0), dim=0).squeeze(0)
                vutils.save_image(ensres, fp=os.path.join(args.out_dir, f"{slice_ID}_output_ens_avg.png"), nrow=1, padding=10)

    except Exception as e:
        logger.log(f"Error during sampling: {e}")
        import traceback
        traceback.print_exc()

    return args.out_dir

In [ ]:
output_dir = sample()

In [ ]:
!ls /kaggle/working/brats/validation/patient_11

In [ ]:
!ls /kaggle/working/out/

In [ ]:
display(Image.open('/kaggle/working/out/brats_val_340_t1_123_w_output0.png'))

In [ ]:
%cd /kaggle/working/out
from IPython.display import FileLink
FileLink('/kaggle/working/out/optsavedmodel000000.pt')

In [ ]:
!tar -cf models.tar /kaggle/working/out

In [ ]:
!ls /kaggle/working/out

In [ ]:
!tar czvf model_tar2.tar.gz /kaggle/working/out

In [ ]:
!ls -alh

In [ ]:
!tar -tvf ./model_tar2.tar.gz

In [ ]:
!nvidia_smi

In [ ]:
sample()

In [ ]:
!ls /kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/

In [ ]:
!ls /kaggle/working/out
from matplotlib import pyplot as plt
from PIL import Image
display(Image.open('/kaggle/working/out/0010016_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010016_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010016.jpg'))

display(Image.open('/kaggle/working/out/0010336_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010336_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010336.jpg'))

display(Image.open('/kaggle/working/out/0010437_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010437_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010437.jpg'))

display(Image.open('/kaggle/working/out/0010494_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010494_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010494.jpg'))

display(Image.open('/kaggle/working/out/0010574_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010574_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010574.jpg'))